In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

tf.logging.set_verbosity(tf.logging.INFO)

In [9]:
import tensorflow as tf

NUM_CLASSES = 228
STARTER_LEARNING_RATE = 0.005
CUT_OFF = 0.184
DECAY_STEPS = 400000
DECAY_RATE = 0.5

def alexnet_model_fn(features, labels, mode):
    """Model function for Alexnet."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    input_layer = tf.convert_to_tensor(features["x"])
    #print("input_layer: {}".format(input_layer.shape))

    conv1 = tf.layers.conv2d(inputs=input_layer,filters=96,kernel_size=[11, 11],strides=4,padding="valid",activation=tf.nn.relu)
    #print("conv1: {}".format(conv1.shape))

    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[3, 3], strides=2, padding='valid')
    #print("pool1: {}".format(pool1.shape))

    conv2 = tf.layers.conv2d(inputs= pool1,filters=256,kernel_size=[5, 5],padding="same",activation=tf.nn.relu)
    #print("conv2: {}".format(conv2.shape))

    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[3, 3], strides=2, padding='valid')
    #print("pool2: {}".format(pool2.shape))

    conv3 = tf.layers.conv2d(inputs=pool2,filters=384,kernel_size=[3, 3],padding="same",activation=tf.nn.relu)
    #print("conv3: {}".format(conv3.shape))

    conv4 = tf.layers.conv2d(inputs=conv3,filters=384,kernel_size=[3, 3],padding="same",activation=tf.nn.relu)
    #print("conv4: {}".format(conv4.shape))

    conv5 = tf.layers.conv2d(inputs=conv4,filters=256,kernel_size=[3, 3],padding="same",activation=tf.nn.relu)
    #print("conv5: {}".format(conv5.shape))

    pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=[3, 3], strides=2,padding='valid')
    #print("pool5: {}".format(pool2.shape))

    pool5_flat = tf.reshape(conv5, [-1, 12*12*256])
    #print("pool5_flat: {}".format(pool5_flat.shape))

    fc6 = tf.layers.dense(inputs=pool5_flat, units=4096, activation=tf.nn.relu)
    #print("dense1: {}".format(fc6.shape))  

    dropout6 = tf.layers.dropout(inputs=fc6, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)
    #print("dropout6: {}".format(dropout6.shape))

    fc7 = tf.layers.dense(inputs=dropout6, units=4096, activation=tf.nn.relu)
    #print("fc7: {}".format(fc7.shape))

    dropout7 = tf.layers.dropout(inputs=fc7, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)
    #print("dropout7: {}".format(dropout7.shape))

    # Logits Layer
    # Input Tensor Shape: [batch_size, 4096]
    # Output Tensor Shape: [batch_size, 228]
    logits = tf.layers.dense(inputs=dropout7, units=NUM_CLASSES)
    #print("logits: {}".format(logits.shape))

    # Generate Predictions
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.cast(tf.sigmoid(logits) >= CUT_OFF, tf.int8, name="class_tensor"),
        # Add `sigmoid_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.sigmoid(logits, name="prob_tensor")
    } 

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    #w_tensor = tf.convert_to_tensor(w)
    #w_tensor = tf.reshape(w_tensor, [-1,228])
    loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=labels, logits=logits)#, weights=w_tensor)

    #loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step = tf.train.get_global_step()
        learning_rate = tf.train.exponential_decay(
            learning_rate=STARTER_LEARNING_RATE, global_step=global_step,
            decay_steps=DECAY_STEPS, decay_rate=DECAY_RATE
        )
        if global_step % DECAY_STEPS == 0:
            tf.logging.info('Learning rate at global step '+str(global_step)+': '+str(learning_rate))
        optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=global_step)
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Customize evaluation metric
    def meanfscore(predictions, labels):
        predictions = tf.reshape(tf.transpose(predictions), [-1])
        labels = tf.convert_to_tensor(labels)
        labels = tf.reshape(tf.transpose(labels), [-1])
        precision_micro, update_op_p = tf.metrics.precision(labels, predictions)
        recall_micro, update_op_r = tf.metrics.recall(labels, predictions)
        f1_mircro = tf.div(tf.multiply(2., tf.multiply(precision_micro, recall_micro)), tf.add(precision_micro, recall_micro), name="eval_tensor")
        return f1_mircro, tf.group(update_op_p, update_op_r)
    
    def precision_micro(predictions, labels):
        predictions = tf.reshape(tf.transpose(predictions), [-1])
        labels = tf.convert_to_tensor(labels)
        labels = tf.reshape(tf.transpose(labels), [-1])
        precision_micro, update_op_p = tf.metrics.precision(labels, predictions)
        return precision_micro, update_op_p
    
    def recall_micro(predictions, labels):
        predictions = tf.reshape(tf.transpose(predictions), [-1])
        labels = tf.convert_to_tensor(labels)
        labels = tf.reshape(tf.transpose(labels), [-1])
        recall_micro, update_op_r = tf.metrics.recall(labels, predictions)
        return recall_micro, update_op_r
    
    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "meanfscore": meanfscore(predictions["classes"], labels),
        "precision_micro": precision_micro(predictions["classes"], labels),
        "recall_micro": recall_micro(predictions["classes"], labels)}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [10]:
import numpy as np
import pandas as pd
import cv2

NUM_CLASSES = 228
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224

def load_images(addrs_list):   
    images = np.empty((len(addrs_list), IMAGE_WIDTH, IMAGE_HEIGHT, 3), dtype=np.float32)
    for i, fpath in enumerate(addrs_list):
        img = cv2.imread(fpath, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (224, 224))
        images[i, ...] = img#.transpose(2, 0, 1) 
        if i % 1000 == 0:
            print('Loading images: {}'.format(i))
    return images

def get_multi_hot_labels(df, index_list):
    label_id = [df['labelId'][i] for i in index_list]
    
    labels_matrix = np.zeros([len(index_list), NUM_CLASSES], dtype=np.uint8())
    
    for i in range(len(label_id)):
        for j in range(len(label_id[i].split(' '))):
            row, col = i, int(label_id[i].split(' ')[j]) - 1
            labels_matrix[row][col] = 1
    
    return labels_matrix

In [31]:
validation_df = pd.read_csv('/home/ec2-user/SageMaker/imat/train.csv')

eval_path_list = validation_df['imagePath']

eval_data = load_images(eval_path_list)
eval_labels = get_multi_hot_labels(validation_df, list(range(validation_df.shape[0])))

Loading images: 0
Loading images: 1000
Loading images: 2000
Loading images: 3000
Loading images: 4000
Loading images: 5000
Loading images: 6000
Loading images: 7000
Loading images: 8000
Loading images: 9000


In [32]:
# Evaluate
multilabel_classifier = tf.estimator.Estimator(
            model_fn=alexnet_model_fn, model_dir="/home/ec2-user/SageMaker/imat/model/multilabel_alexnet_model")
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": eval_data},
            y=eval_labels,
            shuffle=False)
eval_results = multilabel_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/ec2-user/SageMaker/imat/model/multilabel_alexnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe59d16c048>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorf